In [1]:
from ai_data_eng.searching.graph import *
from ai_data_eng.searching.utils import *
from ai_data_eng.searching.searchning import * 
from ai_data_eng.searching.a_star_time_opt import * 
from ai_data_eng.searching.heuristics import WeightedAverageTimeHeuristic
from pathlib import Path

DATA_DIR = Path('../data')

In [2]:
connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv', 
                               usecols=['line', 'departure_time', 'arrival_time', 'start_stop',
       'end_stop', 'start_stop_lat', 'start_stop_lon', 'end_stop_lat',
       'end_stop_lon'])
g = Graph(connection_graph, add_constant_change_time)

C:\Users\julia\AppData\Local\Temp\ipykernel_18168\1218945369.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


In [10]:
from typing import Set
from queue import PriorityQueue


def judge_t_solution(solution, visiting_stops: Set[str]):
    matched_stops = 0 
    conn_time = diff(solution[-1]['arrival_sec'], solution[-1]['departure_sec'])
    line_changes = np.sum([1 for (c1, c2) in zip(solution[1:], solution[:-1]) if c1['line'] != c2['line']])
    
    for conn in solution:
        if conn['start_stop'] in visiting_stops:
            visiting_stops.remove(conn['start_stop'])
            matched_stops += 1
    return conn_time / (matched_stops + 0.01)    
    
def naive_solution(start_stop: str, visiting_stops: List[str], leave_hour: str):
    prev_stop = start_stop
    solution = []
    for stop in visiting_stops:
        _, sub_sol = a_star_time_opt(prev_stop, stop, leave_hour, WeightedAverageTimeHeuristic())
        solution += sub_sol
        prev_stop = stop 
        # add change time 
        leave_hour = sec_to_time(sub_sol[-1]['arrival_sec'])
    solution += a_star_time_opt(prev_stop, start_stop, leave_hour, WeightedAverageTimeHeuristic())[1]
    return solution


def tabu_search(start_stop: str, visiting_stops: List[str], leave_hour: str, criterion: OptimizationType):
    
    dep_time = time_to_normalized_sec(leave_hour)
    prev_stop = start_stop
    solution = []
    for stop in visiting_stops:
        _, sub_sol = a_star_time_opt(prev_stop, stop, leave_hour, WeightedAverageTimeHeuristic())
        solution += sub_sol
        prev_stop = stop 
        # add change time 
        leave_hour = sec_to_time(sub_sol[-1]['arrival_sec'])
    solution += a_star_time_opt(prev_stop, start_stop, leave_hour, WeightedAverageTimeHeuristic())[1]
        
    graph, travel_solution = a_star_time_opt(start_stop, visiting_stops[-1], leave_hour, WeightedAverageTimeHeuristic())
    _, return_solution = a_star_time_opt(visiting_stops[-1], start_stop, sec_to_time(travel_solution[-1]['arrival_sec']), WeightedAverageTimeHeuristic())
    
    solution = travel_solution + return_solution
    print_path(solution)
    cost = judge_t_solution(solution, set(visiting_stops))
    print(cost)

In [12]:
print_path(naive_solution('PL. GRUNWALDZKI', ['most Grunwaldzki', 'Poczta Główna', 'Renoma'], '08:00:00'))

C:\Users\julia\VSCode\AI-and-Data-Engineering\ai_data_eng\searching\searchning.py:93: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',
C:\Users\julia\VSCode\AI-and-Data-Engineering\ai_data_eng\searching\searchning.py:93: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',
C:\Users\julia\VSCode\AI-and-Data-Engineering\ai_data_eng\searching\searchning.py:93: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',
C:\Users\julia\VSCode\AI-and-Data-Engineering\ai_data_eng\searching\searchning.py:93: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connect

PL. GRUNWALDZKI [08:00:00] --- 146 ---> most Grunwaldzki [08:02:00]
most Grunwaldzki [08:02:00] --- D ---> Poczta Główna [08:04:00]
Poczta Główna [08:04:00] --- D ---> GALERIA DOMINIKAŃSKA [08:06:00]
GALERIA DOMINIKAŃSKA [08:06:00] --- D ---> Renoma [08:09:00]
Renoma [08:09:00] --- A ---> Arkady (Capitol) [08:11:00]
Arkady (Capitol) [08:13:00] --- D ---> GALERIA DOMINIKAŃSKA [08:18:00]
GALERIA DOMINIKAŃSKA [08:18:00] --- D ---> Urząd Wojewódzki (Impart) [08:21:00]
Urząd Wojewódzki (Impart) [08:21:00] --- D ---> most Grunwaldzki [08:22:00]
most Grunwaldzki [08:22:00] --- D ---> PL. GRUNWALDZKI [08:24:00]


In [13]:
print_path(naive_solution('Tramwajowa', ['most Grunwaldzki', 'Poczta Główna', 'EPI', 'Sowia', 'Dworzec Główny (Dworcowa)', 'Rynek'], '08:00:00'))

C:\Users\julia\VSCode\AI-and-Data-Engineering\ai_data_eng\searching\searchning.py:93: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',
C:\Users\julia\VSCode\AI-and-Data-Engineering\ai_data_eng\searching\searchning.py:93: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',
C:\Users\julia\VSCode\AI-and-Data-Engineering\ai_data_eng\searching\searchning.py:93: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',
C:\Users\julia\VSCode\AI-and-Data-Engineering\ai_data_eng\searching\searchning.py:93: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connect

Tramwajowa [08:00:00] --- 145 ---> Hala Stulecia [08:02:00]
Hala Stulecia [08:02:00] --- 145 ---> Kliniki - Politechnika Wrocławska [08:04:00]
Kliniki - Politechnika Wrocławska [08:04:00] --- 145 ---> PL. GRUNWALDZKI [08:06:00]
PL. GRUNWALDZKI [08:06:00] --- 145 ---> most Grunwaldzki [08:07:00]
most Grunwaldzki [08:07:00] --- 145 ---> Poczta Główna [08:09:00]
Poczta Główna [08:09:00] --- 145 ---> skwer Krasińskiego [08:11:00]
skwer Krasińskiego [08:11:00] --- 145 ---> DWORZEC GŁÓWNY [08:13:00]
DWORZEC GŁÓWNY [08:14:00] --- 18 ---> DWORZEC AUTOBUSOWY [08:16:00]
DWORZEC AUTOBUSOWY [08:20:00] --- 125 ---> EPI [08:23:00]
EPI [08:24:00] --- 125 ---> DWORZEC AUTOBUSOWY [08:26:00]
DWORZEC AUTOBUSOWY [08:30:00] --- 15 ---> Sanocka [08:32:00]
Sanocka [08:32:00] --- 15 ---> Uniwersytet Ekonomiczny [08:34:00]
Uniwersytet Ekonomiczny [08:34:00] --- 15 ---> Wiśniowa [08:36:00]
Wiśniowa [08:36:00] --- 15 ---> Jaworowa [08:37:00]
Jaworowa [08:37:00] --- 15 ---> Weigla (Szpital) [08:38:00]
Weigla (Szp

In [ ]:
def is_neighbour(dep_time, new_solution):
    return assert_connection_path(dep_time, new_solution)

def naive_search(solution, leave_hour, start_stop, visiting_stops: List[str]):
    prev_stop = start_stop
    new_solution = []
    for stop in visiting_stops:
        _, sub_sol = a_star_time_opt(prev_stop, stop, leave_hour, WeightedAverageTimeHeuristic())
        new_solution = new_solution + sub_sol
        prev_stop = 

def replace_connection(solution, dep_time, stop_from, stop_to):
    
    is_neigh = False
    while not is_neigh:
        graph, travel_solution = a_star_time_opt(stop_from, stop_to, dep_time, WeightedAverageTimeHeuristic())